# Notebook

Format VAERS CSV files, run statistical analysis, transpile charts' JSON.

_Note_:

- VAERS CSV files can be downloaded from [https://vaers.hhs.gov/data/datasets.html](https://vaers.hhs.gov/data/datasets.html).
- Make sure `AllVAERSDataCSVS.zip` exists in the `./csv` directory.

## Format VAERS CSV Files

As a precaution, empty any existing `domestic`/`non_domestic` directories.

In [102]:
import os
import shutil

CSV_PATH = "../csv"

DOMESTIC_CSV_PATH = os.path.join(CSV_PATH, "domestic")

if os.path.exists(DOMESTIC_CSV_PATH):
    shutil.rmtree(DOMESTIC_CSV_PATH)

NON_DOMESTIC_CSV_PATH = os.path.join(CSV_PATH, "non_domestic")

if os.path.exists(NON_DOMESTIC_CSV_PATH):
    shutil.rmtree(NON_DOMESTIC_CSV_PATH)

Certify `./csv` has VAERS CSV files.

In [103]:
!bash ../scripts/unzip.sh

Archive:  ../csv/AllVAERSDataCSVS.zip
  inflating: ../csv/1990VAERSDATA.csv  
  inflating: ../csv/1990VAERSSYMPTOMS.csv  
  inflating: ../csv/1990VAERSVAX.csv  
  inflating: ../csv/1991VAERSDATA.csv  
  inflating: ../csv/1991VAERSSYMPTOMS.csv  
  inflating: ../csv/1991VAERSVAX.csv  
  inflating: ../csv/1992VAERSDATA.csv  
  inflating: ../csv/1992VAERSSYMPTOMS.csv  
  inflating: ../csv/1992VAERSVAX.csv  
  inflating: ../csv/1993VAERSDATA.csv  
  inflating: ../csv/1993VAERSSYMPTOMS.csv  
  inflating: ../csv/1993VAERSVAX.csv  
  inflating: ../csv/1994VAERSDATA.csv  
  inflating: ../csv/1994VAERSSYMPTOMS.csv  
  inflating: ../csv/1994VAERSVAX.csv  
  inflating: ../csv/1995VAERSDATA.csv  
  inflating: ../csv/1995VAERSSYMPTOMS.csv  
  inflating: ../csv/1995VAERSVAX.csv  
  inflating: ../csv/1996VAERSDATA.csv  
  inflating: ../csv/1996VAERSSYMPTOMS.csv  
  inflating: ../csv/1996VAERSVAX.csv  
  inflating: ../csv/1997VAERSDATA.csv  
  inflating: ../csv/1997VAERSSYMPTOMS.csv  
  inflating: ../c

In [104]:
if not os.path.exists(CSV_PATH):
    raise Exception("Error: ../csv does not exist.")

if len(os.listdir(CSV_PATH)) <= 1:
    raise Exception("Error: ../csv is empty.")

#### Define constants.


In [105]:
import datetime

MONTHS = [
    "Jan.",
    "Feb.",
    "Mar.",
    "Apr.",
    "May",
    "June",
    "July",
    "Aug.",
    "Sept.",
    "Oct.",
    "Nov.",
    "Dec.",
]

VAERS_DATASETS = ["DATA", "SYMPTOMS", "VAX"]

YEARS = [str(i) for i in range(1990, int(datetime.date.today().year) + 1)]

#### Define helpers.


Install `requirements.txt`.

In [106]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [107]:
import pandas as pd


def get_data(data, non_domestic, year):
    return pd.read_csv(
        get_data_path(data, non_domestic, year),
        encoding="latin1",
        engine="python",
    )


def get_data_path(data, non_domestic, year):
    return f"{CSV_PATH}/{'non_domestic' if non_domestic else 'domestic'}/{year}VAERS{data}.csv"

Format VAERS CSV files into `domestic` and `nondomestic` directories.


In [108]:
os.mkdir(DOMESTIC_CSV_PATH)

os.mkdir(NON_DOMESTIC_CSV_PATH)

for file in os.listdir(CSV_PATH):
    if not file.startswith("Non") and file.endswith(".csv"):
        os.rename(os.path.join(CSV_PATH, file), os.path.join(DOMESTIC_CSV_PATH, file))

for VAERS_DATASET in VAERS_DATASETS:
    non_domestic_dataset_path = os.path.join(
        CSV_PATH, f"NonDomesticVAERS{VAERS_DATASET}.csv"
    )

    df = pd.read_csv(
        non_domestic_dataset_path,
        encoding="latin1",
        engine="python",
    )

    for YEAR in YEARS:
        year_non_domestic_dataset_path = get_data_path(VAERS_DATASET, True, YEAR)

        if VAERS_DATASET == "DATA":
            year_df = df[df["RECVDATE"].str.contains(YEAR)]
        else:
            data_df = get_data("DATA", True, YEAR)
            vaers_ids = data_df["VAERS_ID"].to_list()
            year_df = df[df["VAERS_ID"].isin(vaers_ids)].drop_duplicates(
                subset=["VAERS_ID"], keep="first"
            )

        year_df.to_csv(
            year_non_domestic_dataset_path,
            encoding="latin1",
            index=False,
        )

    os.remove(non_domestic_dataset_path)

## Run Statistical Analysis

### Define Analysis

#### Ages

Define `analysis_ages`.

In [109]:
def analysis_ages(domestic_data, non_domestic_data):
    ages_0_5_domestic = len(
        domestic_data[domestic_data["AGE_YRS"].lt(5) | domestic_data["CAGE_YR"].lt(5)]
    )
    ages_0_5_non_domestic = len(
        non_domestic_data[
            non_domestic_data["AGE_YRS"].lt(5) | non_domestic_data["CAGE_YR"].lt(5)
        ]
    )
    ages_0_5_total = ages_0_5_domestic + ages_0_5_non_domestic

    ages_6_14_domestic = (
        len(
            domestic_data[
                domestic_data["AGE_YRS"].lt(15) | domestic_data["CAGE_YR"].lt(15)
            ]
        )
        - ages_0_5_domestic
    )
    ages_6_14_non_domestic = (
        len(
            non_domestic_data[
                non_domestic_data["AGE_YRS"].lt(15)
                | non_domestic_data["CAGE_YR"].lt(15)
            ]
        )
        - ages_0_5_non_domestic
    )
    ages_6_14_total = ages_6_14_domestic + ages_6_14_non_domestic

    ages_15_24_domestic = (
        len(
            domestic_data[
                domestic_data["AGE_YRS"].lt(25) | domestic_data["CAGE_YR"].lt(25)
            ]
        )
        - ages_0_5_domestic
        - ages_6_14_domestic
    )
    ages_15_24_non_domestic = (
        len(
            non_domestic_data[
                non_domestic_data["AGE_YRS"].lt(25)
                | non_domestic_data["CAGE_YR"].lt(25)
            ]
        )
        - ages_0_5_non_domestic
        - ages_6_14_non_domestic
    )
    ages_15_24_total = ages_15_24_domestic + ages_15_24_non_domestic

    ages_25_64_domestic = (
        len(
            domestic_data[
                domestic_data["AGE_YRS"].lt(65) | domestic_data["CAGE_YR"].lt(65)
            ]
        )
        - ages_0_5_domestic
        - ages_6_14_domestic
        - ages_15_24_domestic
    )
    ages_25_64_non_domestic = (
        len(
            non_domestic_data[
                non_domestic_data["AGE_YRS"].lt(65)
                | non_domestic_data["CAGE_YR"].lt(65)
            ]
        )
        - ages_0_5_non_domestic
        - ages_6_14_non_domestic
        - ages_15_24_non_domestic
    )
    ages_25_64_total = ages_25_64_domestic + ages_25_64_non_domestic

    ages_unknown_domestic = len(
        domestic_data[domestic_data["AGE_YRS"].isna() & domestic_data["CAGE_YR"].isna()]
    )
    ages_unknown_non_domestic = len(
        non_domestic_data[
            non_domestic_data["AGE_YRS"].isna() & non_domestic_data["CAGE_YR"].isna()
        ]
    )
    ages_unknown_total = ages_unknown_domestic + ages_unknown_non_domestic

    ages_65_plus_domestic = (
        len(domestic_data)
        - ages_0_5_domestic
        - ages_6_14_domestic
        - ages_15_24_domestic
        - ages_25_64_domestic
        - ages_unknown_domestic
    )
    ages_65_plus_non_domestic = (
        len(non_domestic_data)
        - ages_0_5_non_domestic
        - ages_6_14_non_domestic
        - ages_15_24_non_domestic
        - ages_25_64_non_domestic
        - ages_unknown_non_domestic
    )
    ages_65_plus_total = ages_65_plus_domestic + ages_65_plus_non_domestic

    return {
        "0-5": {
            "domestic": ages_0_5_domestic,
            "non_domestic": ages_0_5_non_domestic,
            "total": ages_0_5_total,
        },
        "6-14": {
            "domestic": ages_6_14_domestic,
            "non_domestic": ages_6_14_non_domestic,
            "total": ages_6_14_total,
        },
        "15-24": {
            "domestic": ages_15_24_domestic,
            "non_domestic": ages_15_24_non_domestic,
            "total": ages_15_24_total,
        },
        "25-64": {
            "domestic": ages_25_64_domestic,
            "non_domestic": ages_25_64_non_domestic,
            "total": ages_25_64_total,
        },
        "65+": {
            "domestic": ages_65_plus_domestic,
            "non_domestic": ages_65_plus_non_domestic,
            "total": ages_65_plus_total,
        },
        "unknown": {
            "domestic": ages_unknown_domestic,
            "non_domestic": ages_unknown_non_domestic,
            "total": ages_unknown_total,
        },
    }

#### Outcomes

Define `analysis_outcomes`.

In [110]:
def analysis_outcomes(domestic_data, non_domestic_data):
    outcomes_deaths_domestic = len(domestic_data[domestic_data["DIED"].eq("Y")])
    outcomes_deaths_non_domestic = len(
        non_domestic_data[non_domestic_data["DIED"].eq("Y")]
    )
    outcomes_deats_total = outcomes_deaths_domestic + outcomes_deaths_non_domestic

    outcomes_hospitilizations_domestic = len(
        domestic_data[domestic_data["HOSPITAL"].eq("Y")]
    )
    outcomes_hospitilizations_non_domestic = len(
        non_domestic_data[non_domestic_data["HOSPITAL"].eq("Y")]
    )
    outcomes_hospitilizations_total = (
        outcomes_hospitilizations_domestic + outcomes_hospitilizations_non_domestic
    )

    outcomes_injuries_domestic = (
        len(domestic_data)
        - outcomes_deaths_domestic
        - outcomes_hospitilizations_domestic
    )
    outcomes_injuries_non_domestic = (
        len(non_domestic_data)
        - outcomes_deaths_non_domestic
        - outcomes_hospitilizations_non_domestic
    )
    outcomes_injuries_total = (
        outcomes_injuries_domestic + outcomes_injuries_non_domestic
    )

    return {
        "deaths": {
            "domestic": outcomes_deaths_domestic,
            "non_domestic": outcomes_deaths_non_domestic,
            "total": outcomes_deats_total,
        },
        "hospitilizations": {
            "domestic": outcomes_hospitilizations_domestic,
            "non_domestic": outcomes_hospitilizations_non_domestic,
            "total": outcomes_hospitilizations_total,
        },
        "injuries": {
            "domestic": outcomes_injuries_domestic,
            "non_domestic": outcomes_injuries_non_domestic,
            "total": outcomes_injuries_total,
        },
    }

#### Sexes

Define `analysis_sexes`.

In [111]:
def analysis_sexes(domestic_data, non_domestic_data):
    sexes_f_domestic = len(domestic_data[domestic_data["SEX"].eq("F")])
    sexes_f_non_domestic = len(non_domestic_data[non_domestic_data["SEX"].eq("F")])
    sexes_f_total = sexes_f_domestic + sexes_f_non_domestic

    sexes_m_domestic = len(domestic_data[domestic_data["SEX"].eq("M")])
    sexes_m_non_domestic = len(non_domestic_data[non_domestic_data["SEX"].eq("M")])
    sexes_m_total = sexes_m_domestic + sexes_m_non_domestic

    sexes_unknown_domestic = len(domestic_data) - sexes_f_domestic - sexes_m_domestic
    sexes_unknown_non_domestic = (
        len(non_domestic_data) - sexes_f_non_domestic - sexes_m_non_domestic
    )
    sexes_unknown_total = sexes_unknown_domestic + sexes_unknown_non_domestic

    return {
        "F": {
            "domestic": sexes_f_domestic,
            "non_domestic": sexes_f_non_domestic,
            "total": sexes_f_total,
        },
        "M": {
            "domestic": sexes_m_domestic,
            "non_domestic": sexes_m_non_domestic,
            "total": sexes_m_total,
        },
        "unknown": {
            "domestic": sexes_unknown_domestic,
            "non_domestic": sexes_unknown_non_domestic,
            "total": sexes_unknown_total,
        },
    }

#### Totals

Define `analysis_totals`.

In [112]:
def analysis_totals(domestic_data, non_domestic_data):
    domestic = len(domestic_data)

    non_domestic = len(non_domestic_data)

    return {
        "domestic": domestic,
        "non_domestic": non_domestic,
        "total": domestic + non_domestic,
    }

#### Vaccines

Define `analysis_vaccines`.

In [113]:
def analysis_vaccines(domestic_vax, non_domestic_vax):
    vaccines_list_domestic = domestic_vax["VAX_TYPE"].drop_duplicates().to_list()
    vaccines_list_non_domestic = (
        non_domestic_vax["VAX_TYPE"].drop_duplicates().to_list()
    )
    vaccines_list_total = list(set(vaccines_list_domestic + vaccines_list_non_domestic))

    totals = {}

    for vaccine in vaccines_list_total:
        vaccine_total_domestic = len(domestic_vax[domestic_vax["VAX_TYPE"].eq(vaccine)])
        vaccine_total_non_domestic = len(
            non_domestic_vax[non_domestic_vax["VAX_TYPE"].eq(vaccine)]
        )
        vaccine_total = vaccine_total_domestic + vaccine_total_non_domestic

        totals[vaccine] = {
            "domestic": vaccine_total_domestic,
            "non_domestic": vaccine_total_non_domestic,
            "total": vaccine_total,
        }

    return {
        "lists": {
            "domestic": vaccines_list_domestic,
            "non_domestic": vaccines_list_non_domestic,
            "total": vaccines_list_total,
        },
        "totals": totals,
    }

### Run Analysis

#### Yearly

In [114]:
analysis = {}

years = {}

for YEAR in YEARS:
    year_data = {}

    domestic_year_data = get_data("DATA", False, YEAR)
    domestic_year_symptoms = get_data("SYMPTOMS", False, YEAR)
    domestic_year_vax = get_data("VAX", False, YEAR)

    non_domestic_year_data = get_data("DATA", True, YEAR)
    non_domestic_year_symptoms = get_data("SYMPTOMS", True, YEAR)
    non_domestic_year_vax = get_data("VAX", True, YEAR)

    year_data_months = {}

    for index, MONTH in enumerate(MONTHS):
        month_year_data = {}

        month_domestic_year_data = domestic_year_data[
            domestic_year_data["RECVDATE"]
            .str[:2]
            .eq(str(index + 1) if index + 1 > 9 else f"0{str(index + 1)}")
        ]
        month_domestic_year_vax = domestic_year_vax[
            domestic_year_vax["VAERS_ID"].isin(
                month_domestic_year_data["VAERS_ID"].tolist()
            )
        ]

        month_non_domestic_year_data = non_domestic_year_data[
            non_domestic_year_data["RECVDATE"]
            .str[:2]
            .eq(str(index + 1) if index + 1 > 9 else f"0{str(index + 1)}")
        ]
        month_non_domestic_year_vax = non_domestic_year_vax[
            non_domestic_year_vax["VAERS_ID"].isin(
                month_non_domestic_year_data["VAERS_ID"].tolist()
            )
        ]

        month_year_data_all = {}

        month_year_data_all["ages"] = analysis_ages(
            month_domestic_year_data,
            month_non_domestic_year_data,
        )
        month_year_data_all["outcomes"] = analysis_outcomes(
            month_domestic_year_data,
            month_non_domestic_year_data,
        )
        month_year_data_all["sexes"] = analysis_sexes(
            month_domestic_year_data,
            month_non_domestic_year_data,
        )
        month_year_data_all["totals"] = analysis_totals(
            month_domestic_year_data,
            month_non_domestic_year_data,
        )

        month_year_data_all["vaccines"] = analysis_vaccines(
            month_domestic_year_vax,
            month_non_domestic_year_vax,
        )

        month_year_data["all"] = month_year_data_all

        year_data_months[MONTH] = month_year_data

    year_data["intervals"] = year_data_months

    year_data_all = {}

    year_data_all["ages"] = analysis_ages(domestic_year_data, non_domestic_year_data)
    year_data_all["outcomes"] = analysis_outcomes(
        domestic_year_data, non_domestic_year_data
    )
    year_data_all["sexes"] = analysis_sexes(domestic_year_data, non_domestic_year_data)
    year_data_all["totals"] = analysis_totals(
        domestic_year_data, non_domestic_year_data
    )

    year_data_all["vaccines"] = analysis_vaccines(
        domestic_year_vax, non_domestic_year_vax
    )

    year_data["all"] = year_data_all

    years[YEAR] = year_data

analysis["intervals"] = years

#### All

In [115]:
def merge(a, b):
    results = {}

    for k, v in a.items():
        if k in b.keys():
            if type(v) == int and type(b[k]) == int:
                results[k] = v + b[k]
            elif type(v) == dict and type(b[k]) == dict:
                results[k] = merge(v, b[k])
            elif type(v) == list and type(b[k]) == list:
                results[k] = v + b[k]
            else:
                results[k] = b[k]
        else:
            results[k] = v

    for k, v in b.items():
        if k not in a.keys():
            results[k] = v

    return results

In [116]:
all = {}

for year in YEARS:
    all = merge(all, analysis["intervals"][year]["all"])

analysis["all"] = all

Write `analysis` to `./data/analysis.json`.

In [117]:
import json

DATA_PATH = "../data"

ANALYSIS_DATA_PATH = os.path.join(DATA_PATH, "analysis.json")

if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH)

os.mkdir(DATA_PATH)

with open(ANALYSIS_DATA_PATH, "w") as file:
    json.dump(analysis, file)

## Transpile Charts' JSON

### Ages Chart

In [118]:
def transpile_ages_chart(analysis):
    ages_chart = {}

    ages_chart["unknown"] = {
        "Domestic": analysis["all"]["ages"]["unknown"]["domestic"],
        "Nondomestic": analysis["all"]["ages"]["unknown"]["non_domestic"],
        "Total": analysis["all"]["ages"]["unknown"]["total"],
    }

    ages_chart["age_most"] = max(
        analysis["all"]["ages"].items(),
        key=lambda x: x[1]["total"],
    )[0]

    if ages_chart["age_most"] == "unknown":
        temp_all_ages = analysis["all"]["ages"]
        del temp_all_ages[ages_chart["age_most"]]
        ages_chart["age_most"] = max(
            temp_all_ages.items(),
            key=lambda x: x[1]["total"],
        )[0]

    ages_chart["age_percent"] = (
        analysis["all"]["ages"][ages_chart["age_most"]]["total"]
        / (analysis["all"]["totals"]["total"] - ages_chart["unknown"]["Total"])
    ) * 100

    ages_chart["data"] = [
        {
            "name": "0-5 years",
            "Domestic": analysis["all"]["ages"]["0-5"]["domestic"],
            "Nondomestic": analysis["all"]["ages"]["0-5"]["non_domestic"],
            "Total": analysis["all"]["ages"]["0-5"]["total"],
        },
        {
            "name": "6-14 years",
            "Domestic": analysis["all"]["ages"]["6-14"]["domestic"],
            "Nondomestic": analysis["all"]["ages"]["6-14"]["non_domestic"],
            "Total": analysis["all"]["ages"]["6-14"]["total"],
        },
        {
            "name": "15-24 years",
            "Domestic": analysis["all"]["ages"]["15-24"]["domestic"],
            "Nondomestic": analysis["all"]["ages"]["15-24"]["non_domestic"],
            "Total": analysis["all"]["ages"]["15-24"]["total"],
        },
        {
            "name": "25-64 years",
            "Domestic": analysis["all"]["ages"]["25-64"]["domestic"],
            "Nondomestic": analysis["all"]["ages"]["25-64"]["non_domestic"],
            "Total": analysis["all"]["ages"]["25-64"]["total"],
        },
        {
            "name": "65+ years",
            "Domestic": analysis["all"]["ages"]["65+"]["domestic"],
            "Nondomestic": analysis["all"]["ages"]["65+"]["non_domestic"],
            "Total": analysis["all"]["ages"]["65+"]["total"],
        },
    ]

    return ages_chart

### Outcomes Chart

In [119]:
def transpile_outcomes_chart(analysis):
    outcomes_chart = {}

    outcomes_chart["data"] = [
        {
            "name": "Outcomes",
            "Deaths": analysis["all"]["outcomes"]["deaths"]["total"],
            "Hospitilizations": analysis["all"]["outcomes"]["hospitilizations"][
                "total"
            ],
            "Injuries": analysis["all"]["outcomes"]["injuries"]["total"],
        },
    ]

    outcomes_chart["non_fatal_percent"] = (
        (
            analysis["all"]["outcomes"]["injuries"]["total"]
            + analysis["all"]["outcomes"]["hospitilizations"]["total"]
        )
        / analysis["all"]["totals"]["total"]
    ) * 100

    return outcomes_chart

### Sexes Chart

In [120]:
def transpile_sexes_chart(analysis):
    sexes_chart = {}

    sexes_chart["unknown"] = {
        "Domestic": analysis["all"]["sexes"]["unknown"]["domestic"],
        "Nondomestic": analysis["all"]["sexes"]["unknown"]["non_domestic"],
        "Total": analysis["all"]["sexes"]["unknown"]["total"],
    }

    sexes_chart["sex_most"] = max(
        analysis["all"]["sexes"].items(),
        key=lambda x: x[1]["total"],
    )[0]

    sexes_chart["sex_percent"] = (
        analysis["all"]["sexes"][sexes_chart["sex_most"]]["total"]
        / (analysis["all"]["totals"]["total"] - sexes_chart["unknown"]["Total"])
    ) * 100

    sexes_chart["sex_most"] = "Females" if sexes_chart["sex_most"] == "F" else "Males"

    sexes_chart["data"] = [
        {
            "name": "Female",
            "Domestic": analysis["all"]["sexes"]["F"]["domestic"],
            "Nondomestic": analysis["all"]["sexes"]["F"]["non_domestic"],
            "Total": analysis["all"]["sexes"]["F"]["total"],
        },
        {
            "name": "Male",
            "Domestic": analysis["all"]["sexes"]["M"]["domestic"],
            "Nondomestic": analysis["all"]["sexes"]["M"]["non_domestic"],
            "Total": analysis["all"]["sexes"]["M"]["total"],
        },
    ]

    return sexes_chart

### Totals Chart

In [121]:
def transpile_totals_chart(analysis):
    totals_chart = {}

    totals_chart["interval_most"] = max(
        analysis["intervals"].items(),
        key=lambda x: x[1]["all"]["totals"]["total"],
    )[0]

    totals_chart["interval_total"] = analysis["intervals"][
        totals_chart["interval_most"]
    ]["all"]["totals"]["total"]

    data = []

    for interval in analysis["intervals"].keys():
        data.append(
            {
                "name": interval,
                "Domestic": analysis["intervals"][interval]["all"]["totals"][
                    "domestic"
                ],
                "Nondomestic": analysis["intervals"][interval]["all"]["totals"][
                    "non_domestic"
                ],
                "Total": analysis["intervals"][interval]["all"]["totals"]["total"],
            }
        )

    totals_chart["data"] = data

    return totals_chart

### Vaccines Chart

In [122]:
def transpile_vaccines_chart(analysis):
    vaccines_chart = {}

    vaccines_chart["vaccine_most"] = max(
        analysis["all"]["vaccines"]["totals"].items(),
        key=lambda x: x[1]["total"],
    )[0]

    vaccines_chart["vaccine_percent"] = (
        analysis["all"]["vaccines"]["totals"][vaccines_chart["vaccine_most"]]["total"]
        / analysis["all"]["totals"]["total"]
    ) * 100

    vaccines_chart_children = []

    for vaccine in analysis["all"]["vaccines"]["totals"].keys():
        vaccines_chart_children.append(
            {
                "name": vaccine,
                "Domestic": analysis["all"]["vaccines"]["totals"][vaccine]["domestic"],
                "Nondomestic": analysis["all"]["vaccines"]["totals"][vaccine][
                    "non_domestic"
                ],
                "Total": analysis["all"]["vaccines"]["totals"][vaccine]["total"],
            }
        )

    vaccines_chart["data"] = [
        {
            "name": "Reports",
            "children": vaccines_chart_children,
        }
    ]

    return vaccines_chart

### Transpile Charts

In [123]:
charts = {}

charts["all"] = {}

charts_ages_chart = {}
charts_ages_chart["all"] = transpile_ages_chart(analysis)

charts_outcomes_chart = {}
charts_outcomes_chart["all"] = transpile_outcomes_chart(analysis)

charts_sexes_chart = {}
charts_sexes_chart["all"] = transpile_sexes_chart(analysis)

charts_totals_chart = {}
charts_totals_chart["all"] = transpile_totals_chart(analysis)

charts_vaccines_chart = {}
charts_vaccines_chart["all"] = transpile_vaccines_chart(analysis)

for INTERVAL in analysis["intervals"].keys():
    charts_ages_chart[INTERVAL] = transpile_ages_chart(analysis["intervals"][INTERVAL])

    charts_outcomes_chart[INTERVAL] = transpile_outcomes_chart(
        analysis["intervals"][INTERVAL]
    )

    charts_sexes_chart[INTERVAL] = transpile_sexes_chart(
        analysis["intervals"][INTERVAL]
    )

    charts_totals_chart[INTERVAL] = transpile_totals_chart(
        analysis["intervals"][INTERVAL]
    )

    charts_vaccines_chart[INTERVAL] = transpile_vaccines_chart(
        analysis["intervals"][INTERVAL]
    )

charts["ages_chart"] = charts_ages_chart
charts["outcomes_chart"] = charts_outcomes_chart
charts["sexes_chart"] = charts_sexes_chart
charts["totals_chart"] = charts_totals_chart
charts["vaccines_chart"] = charts_vaccines_chart

Write `charts` to `./data/charts.json`.

In [124]:
CHARTS_DATA_PATH = os.path.join(DATA_PATH, "charts.json")

with open(CHARTS_DATA_PATH, "w") as file:
    json.dump(charts, file)